# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (ie 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - 
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [1]:
# change the cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [3]:
# load other package
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mtl
import warnings
from IPython.display import display_html, HTML
from imp import reload

import sys

sys.path.append("./tool/")

import preprocess
import model

**Background**  
In the last article [Give Best Promotion To Appropriate Customer](https://medium.com/@RayZen/give-best-promotion-to-appropriate-customer-8abb170ca76e), we discuss the company promotion strategy. As we discovered, the different customers agree with different promotion strategy.  Right now, we want to discuss the customers clustering, so that we can find much more detailed information about the customers.

In [4]:
# extract all channels
channels_set = set()
for i in portfolio.channels:
    channels_set.update(set(i))
    
# split the channel into different attribute
for i in channels_set:
    portfolio[i] = portfolio.channels.apply(lambda x: i in x).map({True:1, False: 0})
    
# delete the column channel
portfolio.drop("channels", inplace=True, axis=1)

In [5]:
# get the all kinds of the transcript type value
transcript_type = set()
for i in transcript.value:
    transcript_type.update(set(i.keys()))
    
for i in ["amount", "reward"]:
    transcript[i] = transcript.value.apply(lambda x: x.get(i, np.nan))

transcript["offer_id"] = transcript.value.apply(preprocess.parse_offer_id)

# delete the column value
transcript.drop("value", axis=1, inplace=True)



In [6]:
transcript.head(2)

,event,person,time,amount,reward,offer_id
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [7]:
# check whether the duplicated data is meaning
check_transcript_duplicate = transcript.groupby(["person", "offer_id", "event"]). \
                    size().unstack().reset_index().fillna(0)
check_transcript_duplicate[
    check_transcript_duplicate.apply(lambda x: x["offer completed"] > x["offer received"], axis=1)
]


event,person,offer_id,offer completed,offer received,offer viewed


**Check duplicate data report:**  
After analyzing the duplicated data in the transcript, we find that the duplicate data is meaning. So we need to keep the duplicate data. Because the completed offer amount is not larger than the received offer amount. So we keep the data

In [8]:
# copy the transcript data
data = transcript.copy()

In [9]:
profile.head(2)

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0


In [10]:
data.head(2)

,event,person,time,amount,reward,offer_id
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [11]:
portfolio.head(2)

,difficulty,duration,id,offer_type,reward,email,mobile,web,social
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,1,0,1
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,1,1,1


In [12]:
data = data.merge(portfolio, left_on="offer_id", right_on="id", how="left",
           suffixes=("_transcript", "_offer"))


# delete the duplicate column id
data.drop("id", axis=1, inplace=True)

In [13]:
data.head(2)

,event,person,time,amount,reward_transcript,offer_id,difficulty,duration,offer_type,reward_offer,email,mobile,web,social
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,7.0,bogo,5.0,1.0,1.0,1.0,0.0
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,20.0,10.0,discount,5.0,1.0,0.0,1.0,0.0


In [14]:
# get the different promotion account
transcript_count= data.groupby(["person", "offer_type", "event"]).size(). \
    unstack().reset_index().fillna(0)

In [15]:
transcript_count.head(2)

event,person,offer_type,offer completed,offer received,offer viewed
0,0009655768c64bdeb2e877511632db8f,bogo,1.0,1.0,1.0
1,0009655768c64bdeb2e877511632db8f,discount,2.0,2.0,1.0


In [16]:
# merge the transcript data into the profile
profile = profile.merge(
    preprocess.split_type("person", "offer_type", "offer completed", "_completed",
                          transcript_count).fillna(0),
    left_on="id", right_on="person", how="left"
    )

profile.drop("person", axis=1, inplace=True)

# rename the id name as person
profile.rename({"id": "person"}, axis=1, inplace=True)

In [17]:
def merge_transcript(type_name, suffix, data):
    # merge the rest transcript type
    
    split_result = preprocess.split_type("person", "offer_type", type_name, suffix,
                                  data).fillna(0)
    
    result = profile.merge(split_result, on="person", how="left")
    return result

In [18]:
for column, suffix in zip(["offer received", "offer viewed"], ["_received", "_viewed"]):
    profile = merge_transcript(column, suffix, transcript_count)

In [19]:
# fill the missing value in the different offer status
for i in ["bogo", "discount", "information"]:
    profile.loc[:, profile.columns.str.contains(i)] = \
        profile.loc[:, profile.columns.str.contains(i)].fillna(0)

In [20]:
profile.head()

,age,became_member_on,gender,person,income,bogo_completed,discount_completed,informational_completed,bogo_received,discount_received,informational_received,bogo_viewed,discount_viewed,informational_viewed
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,0.0,2.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,3.0,0.0,0.0,3.0,0.0,1.0,3.0,0.0,1.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN,0.0,0.0,0.0,0.0,3.0,2.0,0.0,2.0,1.0


In [21]:
data.head()

,event,person,time,amount,reward_transcript,offer_id,difficulty,duration,offer_type,reward_offer,email,mobile,web,social
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,7.0,bogo,5.0,1.0,1.0,1.0,0.0
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,20.0,10.0,discount,5.0,1.0,0.0,1.0,0.0
2,offer received,e2127556f4f64592b11af22de27a7932,0,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,10.0,7.0,discount,2.0,1.0,1.0,1.0,0.0
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4,10.0,10.0,discount,2.0,1.0,1.0,1.0,1.0
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0,5.0,bogo,10.0,1.0,1.0,1.0,1.0


In [22]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [23]:
reload(preprocess)

<module 'preprocess' from './tool/preprocess.py'>

In [24]:
profile = profile.merge(preprocess.transaction_count(data, 
                                           "event", "transaction", "person",
                                           {
                                               "index":"person", 
                                               "person":"transaction_num"
                                           }), how="left", on="person")

Right now, we deal with all the transaction data that we will merge the total transaction number into the profile data. Next step, we will do some more things :

* Data wrangling
* Explore the profile data inforamtion

so that we can try to cluster the customer. 

In [25]:
profile.isna().sum()

age                           0
became_member_on              0
gender                     2175
person                        0
income                     2175
bogo_completed                0
discount_completed            0
informational_completed       0
bogo_received                 0
discount_received             0
informational_received        0
bogo_viewed                   0
discount_viewed               0
informational_viewed          0
transaction_num             422
dtype: int64